In [1]:
import pyspark
from pyspark import SparkContext, SparkConf

# **Q:01**

In [2]:
conf = SparkConf().setAppName("Question 1").setMaster("local")
sc = SparkContext(conf = conf)

In [3]:
%%writefile input.txt
Horrid Henry’s hound hunts in the massive Murree mountains. While silly stupid Samuel’s dark dreadful dragon likes to hunt in skies.
Horrid Henry’s hound and Samuel’s dreadful dragon Dany are fast friends and like to hunt and play together. They call themselves fantastic fanciful foursome.

Writing input.txt


In [4]:
Data = sc.textFile("input.txt")

In [5]:
rdd = Data.flatMap(lambda line: line.split(' '))

**(A)**

In [6]:
def consecutive_words(lst, n):
    lst = list(lst)
    for i in range(len(lst) - n + 1):
        yield tuple(lst[i : i+n])

In [7]:
rddA = rdd.glom().flatMap(lambda x: consecutive_words(x, 2)).filter(lambda words: (len(words[0]) > 4) & ((len(words[1]) > 4))).filter(lambda x: x[0][0].lower() == x[1][0].lower()).map(lambda x: (x, 1)).count()

print(rddA)

13


**(B)**

In [8]:
rddB = rdd.glom().flatMap(lambda x: consecutive_words(x, 3)).filter(lambda words: (len(words[0]) > 4) & ((len(words[1]) > 4))).map(lambda x: (x[0][0].upper(), 1)).reduceByKey(lambda x, y: x + y).collect()

for letter, count in rddB:
  print(f"{letter} => {count}")

H => 5
M => 3
W => 1
S => 4
D => 3
T => 1
F => 1


**(C)**

In [9]:
rddC = rdd.glom().flatMap(lambda x: consecutive_words(x, 3)).filter(lambda words: (len(words[0]) > 4) & ((len(words[1]) > 4))).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect()

print(rddC)

[(('Horrid', 'Henry’s', 'hound'), 2), (('Henry’s', 'hound', 'hunts'), 1), (('hound', 'hunts', 'in'), 1), (('massive', 'Murree', 'mountains.'), 1), (('Murree', 'mountains.', 'While'), 1), (('mountains.', 'While', 'silly'), 1), (('While', 'silly', 'stupid'), 1), (('silly', 'stupid', 'Samuel’s'), 1), (('stupid', 'Samuel’s', 'dark'), 1), (('dreadful', 'dragon', 'likes'), 1), (('dragon', 'likes', 'to'), 1), (('skies.', 'Horrid', 'Henry’s'), 1), (('Henry’s', 'hound', 'and'), 1), (('Samuel’s', 'dreadful', 'dragon'), 1), (('dreadful', 'dragon', 'Dany'), 1), (('themselves', 'fantastic', 'fanciful'), 1), (('fantastic', 'fanciful', 'foursome.'), 1)]


In [10]:
sc.stop()

# **Q: 02**

In [11]:
conf = SparkConf().setAppName("Question 2").setMaster("local")
sc = SparkContext(conf = conf)

In [12]:
%%writefile input1.txt
1:1 0 1 0 1 1 1 1 0 1 0
2:1 0 1 0 1 1 1 1 0 0 0
3:1 0 0 0 0 1 1 1 0 1 0
4:1 1 1 1 1 1 1 1 0 1 1

Writing input1.txt


In [13]:
%%writefile Reference.txt
R1:1 0 0 0 1 1 1 1 0 0 0
R2:1 1 1 1 1 1 1 1 1 1 1

Writing Reference.txt


In [14]:
Data = sc.textFile("input1.txt")

Data1 = sc.textFile("Reference.txt")

In [15]:
rdd = Data.map(lambda line: line.split(":")).map(lambda x: (x[0], list(map(int, x[1].split()))))

rdd1 = Data1.map(lambda line: line.split(":")).map(lambda x: (x[0], list(map(int, x[1].split()))))

In [16]:
def jaccard_similarity(input, reference):
  intersection = sum([1 for i, r in zip(input, reference) if i == 1 and r == 1])
  union = sum([1 for i, r in zip(input, reference) if i == 1 or r == 1])
  return intersection / union if union != 0 else 0

In [17]:
rdd2 = rdd.cartesian(rdd1).map(lambda x : (x[0][0], (x[1][0], jaccard_similarity(x[0][1], x[1][1])))).groupByKey().mapValues(lambda ref: max(ref, key = lambda x: x[1])[0]).collect()

for patient, reference in rdd2:
  print(f"{patient} => {reference}")

1 => R1
2 => R1
3 => R1
4 => R2


In [18]:
sc.stop()

# **Q: 03**

In [19]:
conf = SparkConf().setAppName("Question 3").setMaster("local")
sc = SparkContext(conf = conf)

In [20]:
%%writefile input2.txt
Y. Lu -> B. Cao, C. Rego, F. Glover, K. Kiani
B. Cao -> C. Rego, F. Glover, K. Kiani, Y. Lu
C. Rego -> B. Cao, Y. Lu
F. Glover -> B. Cao, Y. Lu
B. Hosseini -> K. Kiani
K. Kiani -> B. Hosseini, B. Cao, Y. Lu

Writing input2.txt


In [21]:
Data = sc.textFile("input2.txt")

In [22]:
rdd = Data.map(lambda line: (line.split("->")[0].strip(), line.split("->")[1].split(", ")))

In [23]:
rdd1 = rdd.flatMapValues(lambda x: x)
rdd2 = rdd1.join(rdd1).map(lambda x: (x[1], x[0])).filter(lambda x: x[0][0] != x[0][1]).map(lambda x: (tuple(sorted(x[0])), x[1])).groupByKey().mapValues(lambda x: list(set(x))).map(lambda x: ((x[0], len(x[1]), x[1]))).collect()

for i in rdd2:
  authors, count, coAuthors = i
  print(f"{authors[0]}, {authors[1]} -> ({count}) {', '. join(coAuthors)}")

 B. Cao, C. Rego -> (1) Y. Lu
 B. Cao, K. Kiani -> (1) Y. Lu
C. Rego, K. Kiani -> (1) Y. Lu
 C. Rego, F. Glover -> (1) B. Cao
 C. Rego, Y. Lu -> (1) B. Cao
F. Glover, Y. Lu -> (1) B. Cao
 B. Hosseini, B. Cao -> (1) K. Kiani
 B. Cao, F. Glover -> (1) Y. Lu
C. Rego, F. Glover -> (1) Y. Lu
F. Glover, K. Kiani -> (2) B. Cao, Y. Lu
 B. Cao, Y. Lu -> (2) F. Glover, C. Rego
 C. Rego, K. Kiani -> (1) B. Cao
K. Kiani, Y. Lu -> (1) B. Cao
 B. Hosseini, Y. Lu -> (1) K. Kiani
B. Cao, Y. Lu -> (1) K. Kiani


In [24]:
sc.stop()